<a href="https://colab.research.google.com/github/risan-raja/21f1000021/blob/main/PA_1_21f1000021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Student Name**: Risan Raja

**Student Roll No**: 21f1000021



### Programming Assignment-1


### Libraries

Stick to `NumPy` and a library such as `matplotlib`, `seaborn` or `plotly` for rendering plots.

In [7]:
# @title
%%html
<style>
.h1_cell {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-size: 200%;
    padding: 10px;
}
.title-word {
  animation: color-animation 2s linear infinite;
}

.title-word-1 {
  --color-1: #DF8453;
  --color-2: #3D8DAE;
  --color-3: #E4A9A8;
}

.title-word-2 {
  --color-1: #DBAD4A;
  --color-2: #ACCFCB;
  --color-3: #17494D;
}

.title-word-3 {
  --color-1: #ACCFCB;
  --color-2: #E4A9A8;
  --color-3: #ACCFCB;
}

.title-word-4 {
  --color-1: #3D8DAE;
  --color-2: #DF8453;
  --color-3: #E4A9A8;
}

@keyframes color-animation {
  0%    {color: var(--color-1)}
  32%   {color: var(--color-1)}
  33%   {color: var(--color-2)}
  65%   {color: var(--color-2)}
  66%   {color: var(--color-3)}
  99%   {color: var(--color-3)}
  100%  {color: var(--color-1)}
}

/* Here are just some visual styles. 🖌 */

.container {
  display: grid;
  place-items: center;
  text-align: center;
  height: 100vh
}

.title {
  font-family: "Arial", sans-serif;
  font-weight: 800;
  font-size: 10vw;
  text-transform: uppercase;
}
</style>
<h1 class ="h1_cell title-word title-word-4" >
    Main Framework to run all problems.
    Modes can be preset to provide different sample universes.
    Only Numpy and Plotly are the external Libraries used.
    </br>
    Please Run the Next Cell before executing any other cell in the notebook.
    </br>
   ⤵
    </h1>


In [8]:
# @title Please Run This Cell
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import warnings

rand = np.random.default_rng()


def make_arm(idx, qstar, qinit=0.0):
    _ = {"id": idx, "q_star": qstar, "q_init": qinit, "activated": 0.0, "qa": qinit}
    return _


def mean_reward(prob):
    mean_rewards_ = np.array([x[0] for x in prob]).mean(axis=0)
    return mean_rewards_


def optimal_arm(prob, n_samples=2000):
    optimal_arms = np.array([x[1] for x in prob]).sum(axis=0) / n_samples
    return optimal_arms


def run_experiment(chunk):
    tmp = [p.simulate() for p in chunk]
    return tmp


class KBandit:
    """
    K-Armed Bandit Problem

    Parameters
    ----------
    steps : int
        No. of epochs
    mode : int
        Mode of the algorithm
    n_arms : int
        No. of arms
    """

    def __init__(self, steps: int = 1000, mode: int = 1, n_arms: int = 10, stationary: bool = True, alpha: float = 0.1,
                 qinit: float = 0.0, ucb_c: int = 2, eps: float = 0.1):
        # No.of Arms
        self.n_arms: int = n_arms
        # Arm_ids
        self.arm_ids = np.arange(self.n_arms, dtype=np.int64)
        self.arms_activated = np.zeros(self.n_arms, dtype=np.int64)
        # Generate Arms
        self.arm_qstar_vals = rand.standard_normal(size=self.n_arms, dtype=np.float64)
        if qinit == 0:
            # self.arms = [make_arm(idx, qstar=i) for idx, i in enumerate(self.arm_qstar_vals)]
            self.arm_qa = np.ones(self.n_arms) * qinit
        else:
            # self.arms: list[dict[str, float]] = [make_arm(idx, qstar=i, qinit=qinit) for idx, i in
            # enumerate(self.arm_qstar_vals)]
            self.arm_qa = np.ones(self.n_arms) * qinit
        # Optimal Action
        self.opt_arm: np.int64 = np.argmax(self.arm_qstar_vals)
        # No. of epochs
        self.steps: int = steps
        # Rewards at each time step
        self.rewards = []
        # Arm chosen at each time step
        self.arm_chosen = []
        # epsilon value
        self.epsilon: float = eps
        # Store variant
        self.mode: int = mode
        # Get the selection function out of the bag
        self.sim_mode = self._modes[self.mode]
        self.ucb_c: int = ucb_c
        if self.mode == 3:
            if ucb_c is None:
                raise ValueError("ucb_c cannot be None")
            else:
                self.ucb_c = ucb_c
        self._is_started: bool = False
        self._time: int = 0
        self.stationary: bool = stationary
        self.alpha: float = alpha
        self.qinit: float = qinit

    @property
    def _modes(self):
        return {1: self.greedy, 2: self.greedy_eps, 3: self.ucb_arm}

    def set_mode(self, mode: int):
        """
        Set the mode of the algorithm to reuse the same object for different modes.

        Parameters
        ----------
        mode : int
        """
        if self._is_started:
            raise ValueError("Cannot change the mode after the simulation has started, The samples have been worked."
                             " Please create a new object for different modes. or reset the object"
                             f" New Mode Given: {mode}, Current Mode: {self.mode}")
        else:
            self.mode = mode
            self.sim_mode = self._modes[self.mode]
            if self.mode == 3:
                if self.ucb_c is None:
                    raise ValueError("ucb_c cannot be None")
                else:
                    self.ucb_c = self.ucb_c

    def set_qinit(self, qinit: float):
        """
        Set the qinit value of the algorithm to reuse the same object for different qinit values.

        Parameters
        ----------
        qinit : float
        """
        if self._is_started:
            raise ValueError("Cannot change the qinit after the simulation has started, The samples have been worked.")
        else:
            self.arm_qa = np.ones(self.n_arms) * qinit

    def update_arm_qa(self, arm_id):
        """
        Update the Arm's Value Function

        Parameters
        ----------
        arm_id : int
        """
        arm_reward = rand.normal(loc=self.arm_qstar_vals[arm_id], scale=1)
        # self.arms[arm_id]["activated"] += 1
        self.arms_activated[arm_id] += 1
        total_activation = self.arms_activated[arm_id]
        if self.stationary:
            self.arm_qa[arm_id] += (1 / total_activation) * (arm_reward - self.arm_qa[arm_id])
        else:
            self.arm_qa[arm_id] += self.alpha * (arm_reward - self.arm_qa[arm_id])
        self._time += 1
        self.update_arm_chosen(arm_id)
        self.rewards.append(arm_reward)

    @property
    def random_arm(self) -> np.int64:
        """
        Randomly select an arm

        Returns
        -------
        int
        """
        return rand.choice(self.arm_ids)

    def greedy(self) -> np.int64:
        """
        Greedy Algorithm - Select the optimal arm with probability 1

        Returns
        -------
        int
        """
        # top_ = max(self.arms, key=itemgetter("qa"))["id"]
        top_ = np.argmax(self.arm_qa)
        return top_

    def greedy_eps(self) -> np.int64:
        """
        Epsilon Greedy Algorithm - Select the optimal arm with probability 1 - eps

        Returns
        -------
        int

        """
        if rand.random() < self.epsilon:
            return self.random_arm
        else:
            return self.greedy()

    def update_arm_chosen(self, arm_id: np.int64):
        """
        Update whether the arm chosen at each time step is the optimal arm or not

        Parameters
        ----------
        arm_id : int
        """
        if arm_id == self.opt_arm:
            self.arm_chosen.append(True)
        else:
            self.arm_chosen.append(False)

    def ucb_arm(self):
        """
        Upper Confidence Bound Algorithm

        Returns
        -------
        int
        """
        # activated = np.array([i["activated"] for i in self.arms])
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            ucb_vals = self.arm_qa + self.ucb_c * np.sqrt(np.log(self._time) / self.arms_activated)
        return np.argmax(ucb_vals)

    def interact(self, arm_id: np.int64):
        """
        Interact with the environment

        Parameters
        ----------
        arm_id : int
        """
        self.update_arm_qa(arm_id)

    def simulate(self) -> tuple[float, bool]:
        """
        Simulate the Bandit Problem

        Returns
        -------
        list[float]
        """
        self._is_started = True
        arm_selector = self.sim_mode
        self.interact(self.random_arm)
        for i in range(self.steps - 1):
            self.interact(arm_selector())
        return self.rewards, self.arm_chosen

    def reset(self):
        self._is_started = False
        self._time = 0
        self.arm_qa = np.ones(self.n_arms) * self.qinit
        self.rewards = []
        self.arm_chosen = []


def generate_samples(n_samples: int, n_arms: int, mode: int, n_steps: int, qinit: float = 0.0, eps: float = 0.1,
                     ucb_c: int = 2, alpha: float = 0.1, stationary: bool = True) -> list[KBandit]:
    """
    Generate n samples of the Bandit Problem

    Parameters
    ----------
    n_samples : (int) No. of samples
    n_arms : (int) Arms with which the bandit problem is to be simulated
    mode : (int) Mode of the algorithm
    n_steps : (int) No. of epochs
    qinit : (float) Initial value of the action value function
    eps : (float) Epsilon value for epsilon greedy algorithm
    ucb_c : (int) UCB C value for UCB algorithm
    alpha : (float) Learning rate for non-stationary problems
    stationary : (bool) Stationary or Non-Stationary Problem


    Returns
    -------
    list[KBandit]
    """
    return [KBandit(n_arms=n_arms, steps=n_steps, mode=mode, qinit=qinit, eps=eps, ucb_c=ucb_c, alpha=alpha,
                    stationary=stationary) for _ in range(n_samples)]


# redundant

def check_multiple_values_passed(arg, default):
    if type(arg) is list:
        if len(arg) > 1:
            warnings.warn("Multiple values passed for a parameter, only the first value will be considered, "
                          "In case of multiple values, please enable the check_<parameter>")
            return [arg[0]]
        else:
            return arg
    elif arg is None:
        return [default]
    else:
        return [arg]


def check_option_validity(option, vals, option_name, vals_name):
    if option:
        if type(vals) is not list:
            raise ValueError(f"{vals_name} should be a list. if {option_name} is enabled")
        elif len(vals) == 1:
            raise ValueError(f"{vals_name} should have more than 1 value. Please disable {option_name} for single run.")
        else:
            return True


class BanditProblem:
    """
    The Bandit Problem Class

    ---
    This class is used to generate the bandit problem for different modes and different parameters.
    The methods generate_stationary_cases and generate_non_stationary_cases are used to generate the bandit problems
    and are intended to be used with the fast_simulator and slow_simulator methods.
    The fast_simulator and slow_simulator methods are used to simulate the bandit problems in parallel.

    """

    # noinspection PyDefaultArgument
    def __init__(self, n_arms, n_epochs, modes: list[int], n_samples: int, qinit_vals=None, alpha_vals=None,
                 ucb_c_vals=None, check_multiple_eps=True, check_multiple_init=False, check_multiple_alpha=False,
                 check_multiple_ucb_c=False, eps_vals=[0.1, 0.01]):
        self.n_arms = n_arms
        self.n_epochs = n_epochs
        self.n_samples = n_samples
        self.kbandits_master = generate_samples(n_samples=self.n_samples, n_arms=self.n_arms, mode=1,
                                                n_steps=self.n_epochs)
        self.modes = modes
        self.check_multiple_init = check_multiple_init
        self.check_multiple_alpha = check_multiple_alpha
        self.check_multiple_ucb_c = check_multiple_ucb_c
        if check_option_validity(check_multiple_init, qinit_vals, "check_multiple_init", "qinit_vals"):
            self.qinit_vals = qinit_vals
        else:
            self.qinit_vals = check_multiple_values_passed(qinit_vals, 0)
        if check_option_validity(check_multiple_alpha, alpha_vals, "check_multiple_alpha", "alpha_vals"):
            self.alpha_vals = alpha_vals
        else:
            self.alpha_vals = check_multiple_values_passed(alpha_vals, 0.1)
        if check_option_validity(check_multiple_ucb_c, ucb_c_vals, "check_multiple_ucb_c", "ucb_c_vals"):
            self.ucb_c_vals = ucb_c_vals
        else:
            self.ucb_c_vals = check_multiple_values_passed(ucb_c_vals, 2)
        if check_option_validity(check_multiple_eps, eps_vals, "check_multiple_eps", "eps_vals"):
            self.eps_vals = eps_vals
        else:
            self.eps_vals = check_multiple_values_passed(eps_vals, 0.1)
        self.stationary_cases = None
        self.non_stationary_cases = None
        self.result_storage = {"stationary": {"mean_rewards": [], "optimal_arms_chosen": []}}

    def reset_master(self):
        """
        Deprecated

        Returns
        -------

        """
        for i in range(len(self.kbandits_master)):
            self.kbandits_master[i].reset()

    def generate_stationary_cases(self):
        """
        Simulate the stationary cases

        Returns
        -------
        generator

        """
        for i in self.qinit_vals:
            for m in self.modes:
                if m == 1:
                    tmp = generate_samples(n_samples=self.n_samples, n_arms=self.n_arms, mode=1, n_steps=self.n_epochs,
                                           qinit=i)
                    yield tmp
                elif m == 2:
                    for e in self.eps_vals:
                        tmp = generate_samples(n_samples=self.n_samples, n_arms=self.n_arms, mode=2,
                                               n_steps=self.n_epochs, qinit=i, eps=e)
                        yield tmp
                elif m == 3:
                    for c in self.ucb_c_vals:
                        tmp = generate_samples(n_samples=self.n_samples, n_arms=self.n_arms, mode=3,
                                               n_steps=self.n_epochs, qinit=i, ucb_c=c)
                        yield tmp

    def generate_non_stationary_cases(self):
        """
        Generate the non-stationary cases

        Returns
        -------
        generator
        """
        for alpha in self.alpha_vals:
            for i in self.qinit_vals:
                for m in self.modes:
                    if m == 1:
                        tmp = generate_samples(n_samples=self.n_samples, n_arms=self.n_arms, mode=1,
                                               n_steps=self.n_epochs, stationary=False, qinit=i, alpha=alpha)
                        yield tmp
                    elif m == 2:
                        for e in self.eps_vals:
                            tmp = generate_samples(n_samples=self.n_samples, n_arms=self.n_arms, mode=1,
                                                   n_steps=self.n_epochs, stationary=False, qinit=i, alpha=alpha, eps=e)
                            yield tmp
                    elif m == 3:
                        for c in self.ucb_c_vals:
                            tmp = generate_samples(n_samples=self.n_samples, n_arms=self.n_arms, mode=1,
                                                   n_steps=self.n_epochs, stationary=False, qinit=i, alpha=alpha,
                                                   ucb_c=c)
                            yield tmp

    @staticmethod
    def fast_simulator(probs):
        """
        Simulate all Bandit Problem in parallel.

        Parameters
        ----------
        probs

        Returns
        -------
        list[list[tuple[float, bool]]]

        """
        all_results = []
        with ProcessPoolExecutor() as executor:
            results = executor.map(run_experiment, probs)
            for i in results:
                all_results.append(i)
        return all_results

    @staticmethod
    def slow_simulator(probs):
        """
        Simulate each Bandit Problem in parallel chunks of 4.

        Parameters
        ----------
        probs

        Returns
        -------
        list[list[tuple[float, bool]]]
        """
        all_results = []
        with ProcessPoolExecutor() as executor:
            for prob in probs:
                prob_chunks = np.array_split(prob, 10)
                result = executor.map(run_experiment, prob_chunks)
                final_result = []
                for i in result:
                    final_result += i
                all_results.append(final_result)
        return all_results

    def simulate_stationary_cases(self, fast=True):
        """
        Simulate the stationary cases

        Parameters
        ----------
        fast

        Returns
        -------
        list[list[tuple[float, bool]]]
        """
        self.stationary_cases = self.generate_stationary_cases()
        if fast:
            return self.fast_simulator(self.stationary_cases)
        else:
            return self.slow_simulator(self.stationary_cases)

    def process_results(self, results):
        """
        Process the results of the simulation

        Parameters
        ----------
        results

        Returns
        -------
        list[list[tuple[float, bool]]]
        """
        mean_rewards = [mean_reward(prob) for prob in results]
        optimal_arms_chosen = [optimal_arm(prob) for prob in results]
        self.result_storage["stationary"]["mean_rewards"] = mean_rewards
        self.result_storage["stationary"]["optimal_arms_chosen"] = optimal_arms_chosen
        return mean_rewards, optimal_arms_chosen

    def plot_results(self, modes: list[float] = None):
        """
        Plot the results of the simulation

        Returns
        -------

        """
        modes_store = [1, 2.1, 2.01, 3]
        labels = ['\u03b5=0 (greedy)', '\u03b5=0.1', '\u03b5=0.01', 'UCB c=2']
        c = ['green', 'blue', 'red', 'grey']
        selected_idx = [modes_store.index(i) for i in modes]
        mean_rewards = [self.result_storage["stationary"]["mean_rewards"][i] for i in selected_idx]
        optimal_arms_chosen = [self.result_storage["stationary"]["optimal_arms_chosen"][i] for i in selected_idx]
        labels = [labels[i] for i in selected_idx]
        c = [c[i] for i in selected_idx]
        fig = make_subplots(rows=1, cols=2, subplot_titles=("Average Reward / Steps", "Optimal Action / Steps"),
                            horizontal_spacing=0.15)  # Mean Rewards
        for rew, label, colr in zip(mean_rewards, labels, c):
            fig.add_trace(go.Scatter(y=rew, name=label, line=dict(color=colr), legendgroup=label), row=1, col=1)
        fig.update_yaxes(title_text="Average Reward", row=1, col=1)
        fig.update_xaxes(title_text="Steps", row=1, col=1)
        # Optimal Arms Chosen
        for opt, label, colr in zip(optimal_arms_chosen, labels, c):
            fig.add_trace(go.Scatter(y=opt, name=label, line=dict(color=colr), showlegend=False, legendgroup=label),
                          row=1, col=2)
        fig.update_yaxes(title_text="Optimal Action", row=1, col=2)
        fig.update_xaxes(title_text="Steps", row=1, col=2)
        plot_title = (f"<b>Arms Test Bed - <span style='color:green'>{self.n_arms}</span> Arms, "
                      f"<span style='color:green'>{self.n_samples}</span> Samples,"
                      f" <span style='color:green'>{self.n_epochs}</span> Steps</b></br>")
        title_options = dict(text=plot_title, x=0.5, xanchor='center', y=0.95, yanchor='top',
                             font=dict(size=20, family="Raleway"))
        fig.update_layout(height=500, title=title_options)
        fig.write_html(f"plot_results_modes{('_').join([str(i) for i in modes])}.html", include_plotlyjs="cdn",
                       full_html=True)
        fig.show()


## Problem 1 [65 Marks]

🇷 🇺 🇳 </br>   🇹 🇭 🇪 </br>    🇳 🇪 ❌ 🇹  </br>   🇨 🇪 🇱 🇱

To generate data for questions 1 and 2



In [13]:
def get_minimal_bp():
    bp_ = BanditProblem(n_arms=10, n_epochs=2000, modes=[1, 2, 3], n_samples=2000)
    return bp_

bp = get_minimal_bp()
results = bp.simulate_stationary_cases(fast=True)
processed_results = bp.process_results(results)

# __Answer 1__

In [11]:
bp.plot_results([1,2.1,2.01])

### <h1 style="color:green"> Observations from problem-1 </span>
---

    - The ε=0 greedy method on average performs the worst in comparison to the other methods.
    - ε = 0.1 stops improving after certain steps.
    - ε = 0.01 shows the most promise in results to find the better action/reward.
    - ε = 0.1, although does more exploration, its randomness in choosing 10% of its actions not based on action value heuristic concludes to be detrimental.

## Problem 2 [35 Marks]

Run the UCB algorithm on the ten-armed testbed. Plot the learning curve for UCB and compare it with $\epsilon$-greedy for $\epsilon = 0.1$. Reproduce the following learning curves averaged over 2000  bandits for 1000 timesteps:

In [12]:
# @title
bp.plot_results([2.1,3])

### Observations from problem-2
---
    - On closer observation, UCB does provide more performance and also shows that it's not prone to get stuck in a local minima, compared to ε=0.1 greedy method.
    - UCB explores a number of actions to restrict its search space initially and exploits the best actions.
    - Whereas the ε-greedy method continues to operate with a certain degree of randomness in all horizons.
    

## Bonus Problem [10 Marks]

Solve problems 1 and 2 for $10,000$ arms. Appropriately choose number of samples/epochs for each problem.

## __PS: The following command can take ages to run Please Look below to find the rendered graph obtained locally.__

In [14]:
# write your code here for the bonus problem
bp_2 = BanditProblem(n_arms=10000, n_epochs=30000, modes=[1, 2, 3], n_samples=400)
results = bp_2.simulate_stationary_cases(fast=False)
processed_results = bp_2.process_results(results)
bp_2.plot_results([1,2.1,2.01])

In [15]:
bp_2.plot_results([2.1,3])

### Observations from bonus problem

Enter your observations